In [2]:
import os
import c3d
import glob
import math
import numpy as np
import mayavi as mlab
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D

%matplotlib qt

In [2]:
# example 3d plotting
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x = [1,2,3,4,5,6,7,8,9,10]
y = [5,6,2,3,13,4,1,2,4,8]
z = [2,3,3,3,5,7,9,11,9,10]

ax.scatter(x, y, z, c='r', marker='o') # scatter plot

ax.set_xlabel('x_axis')
ax.set_ylabel('y_axis')
ax.set_zlabel('z_axis')

plt.show()

In [3]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
X, Y, Z = [1,2,3,4,5,6,7,8,9,10],[5,6,2,3,13,4,1,2,4,8],[2,3,3,3,5,7,9,11,9,10]
ax.plot(X, Y, Z, label='parametric curve')

plt.show()

In [4]:
mpl.rcParams['legend.fontsize'] = 10

fig = plt.figure()
ax = fig.gca(projection='3d')
theta = np.linspace(-4 * np.pi, 4 * np.pi, 100)
z = np.linspace(-2, 2, 100)
r = z**2 + 1
x = r * np.sin(theta)
y = r * np.cos(theta)
ax.plot(x, y, z, label='parametric curve')
ax.legend()

plt.show()

In [5]:
# try to read c3d files
cd3data_path = os.path.join('data', 'motion_data', 'files_motions_589')
all_files = glob.glob(cd3data_path+'/**/*[0-9].c3d', recursive=True)
for files in all_files: 
    print(files)

# read c3d file
reader = c3d.Reader(open(all_files[0], 'rb'))

data/motion_data/files_motions_589/pour02.c3d
data/motion_data/files_motions_589/pour04.c3d
data/motion_data/files_motions_589/pour03.c3d
data/motion_data/files_motions_589/pour01.c3d
data/motion_data/files_motions_589/pour05.c3d
data/motion_data/files_motions_589/pour06.c3d


### See information about marker points [here](https://motion-database.humanoids.kit.edu/marker_set/)

In [17]:
# reader.point_used
point_labels = reader.point_labels
# print(type(point_labels))
reader.point_labels

['small_cup:sc_01               ',
 'small_cup:sc_02               ',
 'small_cup:sc_03               ',
 'small_cup:sc_04               ',
 'bottle:shampoo_01             ',
 'bottle:shampoo_02             ',
 'bottle:shampoo_03             ',
 'bottle:shampoo_04             ',
 'Table:table_1                 ',
 'Table:table_2                 ',
 'Table:table_3                 ',
 'Table:table_4                 ',
 'subjX:RPSI                    ',
 'subjX:LPSI                    ',
 'subjX:L3                      ',
 'subjX:STRN                    ',
 'subjX:T10                     ',
 'subjX:C7                      ',
 'subjX:CLAV                    ',
 'subjX:RBAK                    ',
 'subjX:LBAK                    ',
 'subjX:LSHO                    ',
 'subjX:LUPA                    ',
 'subjX:LAEL                    ',
 'subjX:LAOL                    ',
 'subjX:LWTS                    ',
 'subjX:LWPS                    ',
 'subjX:LFRA                    ',
 'subjX:LIFD        

In [67]:
# the marker points that we need
# left_arm_markers = ['LSHO', 'LUPA', 'LAEL', 'LAOL', 'LWTS', 'LWPS', 'LIFD']
# right_arm_markers = ['RSHO', 'RUPA', 'RAEL', 'RAOL', 'RWTS', 'RWPS', 'RIFD']
left_arm_points_ids = list(range(21,31))
right_arm_points_ids = list(range(31,41))
print(len(right_arm_points_ids))

10


In [68]:
right_arm_points = []
for point_id in right_arm_points_ids:
    point_marker = []
    for i, points, analog in reader.read_frames():
        point_marker.append(points[point_id][:3])
    right_arm_points.append(point_marker)

In [134]:
points_array = np.array(right_arm_points)
print(np.shape(points_array))

(10, 831, 3)


In [70]:
# visualize a frame
frame = points_array[:,0,:]
x_data = frame[:,0]
y_data = frame[:,1]
z_data = frame[:,2]

In [143]:
# plot joint positions
fig = plt.figure()
aax = fig.gca(projection='3d')

aax.scatter(x_data, y_data, z_data)
# aax.plot(x_data, y_data, z_data)

aax.set_xlabel('x_axis')
aax.set_ylabel('y_axis')
aax.set_zlabel('z_axis')

aax.azim = 180
aax.elev = 10
aax.autoscale()
# aax.legend()

# %matplotlib qt
plt.show()

### we are going to try to make animations of the arm movement

#### Example:

In [112]:
"""
============
3D animation
============

A simple example of an animated plot... In 3D!
"""
# import numpy as np
# import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
# import matplotlib.animation as animation


def Gen_RandLine(length, dims=2):
    """
    Create a line using a random walk algorithm

    length is the number of points for the line.
    dims is the number of dimensions the line has.
    """
    lineData = np.empty((dims, length))
    lineData[:, 0] = np.random.rand(dims)
    for index in range(1, length):
        # scaling the random numbers by 0.1 so
        # movement is small compared to position.
        # subtraction by 0.5 is to change the range to [-0.5, 0.5]
        # to allow a line to move backwards.
        step = ((np.random.rand(dims) - 0.5) * 0.1)
        lineData[:, index] = lineData[:, index - 1] + step

    return lineData


def update_lines(num, dataLines, lines):
    for line, data in zip(lines, dataLines):
        # NOTE: there is no .set_data() for 3 dim data...
        line.set_data(data[0:2, :num])
        line.set_3d_properties(data[2, :num])
    return lines

# Attaching 3D axis to the figure
fig = plt.figure()
ax = p3.Axes3D(fig)

# Fifty lines of random 3-D lines
data = [Gen_RandLine(25, 3) for index in range(50)]

# Creating fifty line objects.
# NOTE: Can't pass empty arrays into 3d version of plot()
lines = [ax.plot(dat[0, 0:1], dat[1, 0:1], dat[2, 0:1])[0] for dat in data]

# Setting the axes properties
ax.set_xlim3d([0.0, 1.0])
ax.set_xlabel('X')

ax.set_ylim3d([0.0, 1.0])
ax.set_ylabel('Y')

ax.set_zlim3d([0.0, 1.0])
ax.set_zlabel('Z')

ax.set_title('3D Test')

# Creating the Animation object
line_ani = animation.FuncAnimation(fig, update_lines, 25, fargs=(data, lines),
                                   interval=50, blit=False)

plt.show()


In [113]:
np.shape([Gen_RandLine(25, 3) for index in range(50)])

(50, 3, 25)

In [86]:
frame = points_array[:,0,:]
x_data = frame[:,0]
y_data = frame[:,1]
z_data = frame[:,2]

print(np.shape(x_data))

(10,)


In [135]:
np.shape(points_array[:,:20,:])

(10, 20, 3)

In [161]:
def frame_gen():
    total_frames = np.shape(points_array)[0]
    t = 0
    while t < total_frames:
        frame = points_array[:,t,:]
        x_data = frame[:,0]
        y_data = frame[:,1]
        z_data = frame[:,2]
        t += 1
        yield x_data, y_data, z_data

def update_lines(num, dataLines, lines):
    for line, data in zip(lines, dataLines):
        # NOTE: there is no .set_data() for 3 dim data...
        line.set_data(data[0:2, :num])
        line.set_3d_properties(data[2, :num])
    return lines

# Attaching 3D axis to the figure
fig = plt.figure()
# c3d_plot = p3.Axes3D(fig)

c3d_plot = fig.gca(projection='3d')



# samples of 3d points
data = np.transpose(points_array[:,:20,:], (1,2,0))

# Setting the axes properties
c3d_plot.set_xlim3d([0.0, 200.0])
c3d_plot.set_xlabel('X')

c3d_plot.set_ylim3d([200.0, 600.0])
c3d_plot.set_ylabel('Y')

c3d_plot.set_zlim3d([500.0, 1500.0])
c3d_plot.set_zlabel('Z')

c3d_plot.set_title('C3D Arm Points 3D')

# Creating fifty line objects.
# NOTE: Can't pass empty arrays into 3d version of plot()
lines = [c3d_plot.plot(dat[0, 0:1], dat[1, 0:1], dat[2, 0:1])[0] for dat in data]
# lines = [c3d_plot.scatter3D(dat[0, 0:1], dat[1, 0:1], dat[2, 0:1]) for dat in data]

# Creating the Animation object
line_ani = animation.FuncAnimation(fig, update_lines, 10, fargs=(data, lines),
                                   interval=50, blit=False)

plt.show()

In [152]:
data[0]

array([[   88.78457642,    67.27487183,   116.42889404,   180.6335907 ,
           -1.45498979,    52.49711227,    81.7328949 ,    15.1585865 ,
          -11.72694016,    46.36584854],
       [  233.95379639,   294.85906982,   427.38742065,   403.38809204,
          477.0796814 ,   521.84283447,   464.51306152,   489.92703247,
          501.51455688,   530.55310059],
       [ 1437.06311035,  1386.28613281,  1153.73596191,  1128.96691895,
          924.83496094,   922.40765381,  1071.06811523,   763.5       ,
          837.20666504,   847.394104  ]])

In [142]:
update_lines(10, data, lines)

In [110]:
gen = gen_points_array(points_array)

array([[   88.78457642,   233.95379639,  1437.06311035],
       [   67.27487183,   294.85906982,  1386.28613281],
       [  116.42889404,   427.38742065,  1153.73596191],
       [  180.6335907 ,   403.38809204,  1128.96691895],
       [   -1.45498979,   477.0796814 ,   924.83496094],
       [   52.49711227,   521.84283447,   922.40765381],
       [   81.7328949 ,   464.51306152,  1071.06811523],
       [   15.1585865 ,   489.92703247,   763.5       ],
       [  -11.72694016,   501.51455688,   837.20666504],
       [   46.36584854,   530.55310059,   847.394104  ]])